# Dependency Injection in FastAPI

- authentication system
- query parameters' validator
- rate-limiter

In FastAPI a dependency injection can even call another one recursively

## What is dependency injection?

Generally speaking, dependency injection is a system able to automatically instantiate objects and the ones they depend on.

In [1]:
from fastapi import FastAPI, Header

app = FastAPI()


@app.get("/")
async def header(user_agent: str = Header(...)):
    return {
        "user_agent": user_agent
    }

In [3]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

In [4]:
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [38307]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60576 - "POST /posts HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:60580 - "POST / HTTP/1.1" 405 Method Not Allowed
INFO:     127.0.0.1:60580 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:60617 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60617 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60618 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [38307]


![uk6VMU](https://raw.githubusercontent.com/azataiot/images/master/2021/12/20/uk6VMU.png)

Internally, the Header function has some logic to automatically get the request object, check for the required header, return its value, or raise an error if it's not present. From the developer's perspective, however, we don't know how it handled the required objects for this operation: we just ask for the value we need. That's dependency injection.

## Creating and using a function dependency
In FastAPI, a dependency can be defined either as a function or as a callable class

In [5]:
from fastapi import Depends

app = FastAPI()
from typing import Tuple


async def pagination(skip: int = 0, limit: int = 10) -> Tuple[int, int]:
    return skip, limit


@app.get("/items")
async def list_items(p: Tuple[int, int] = Depends(pagination)):
    skip, limit = p
    return {
        "skip": skip,
        "limit": limit
    }

In [6]:
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [38307]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60932 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:60932 - "GET /items/ HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:60932 - "GET /items HTTP/1.1" 200 OK
INFO:     127.0.0.1:60938 - "GET /items/?skip=5 HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:60938 - "GET /items?skip=5 HTTP/1.1" 200 OK
INFO:     127.0.0.1:60944 - "GET /items/?limit=30&skip=5 HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:60944 - "GET /items?limit=30&skip=5 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [38307]


![CleanShot2021-12-20at18.50.00](https://raw.githubusercontent.com/azataiot/images/master/2021/12/20/CleanShot%202021-12-20%20at%2018.50.00.png)

## Creating an using a parameterized dependency with a class

## Using dependency at a path, router, and global level